In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import math

import statsmodels.api as sm
from statsmodels.sandbox.regression.gmm import GMM
from statsmodels.base.model import GenericLikelihoodModel

from scipy.stats import norm
from scipy.stats import multivariate_normal

In [4]:
#load data into memory
data = pd.DataFrame(data = np.genfromtxt('sim3.dat', delimiter='  '), columns=['i','t','y_t','p_t'])

#set up lag
shift_data = data[['i','y_t','p_t']]
shift_data['t'] = data['t'] + 1
data = data.merge(shift_data,how='left',on=['i','t'],suffixes=['','-1'])
data = data.dropna()

# Part a

# Part b

# Part c

# Part d

In [6]:
NSIM = 30
T = int(data.groupby('i').count().max()['t'])#data is nicely formatted
I = len(data.i.unique())

print I, T

alpha = np.random.normal(0,1 ,(NSIM, I))
alpha = alpha.reshape( (1, NSIM, I) )
alpha = np.tile(alpha, (T, 1,1))
print alpha.shape

100 19
(19, 30, 100)


In [7]:
#index = data[['i']]
#alpha = np.array([data.i.unique()]).transpose()
#sims = np.random.normal(0,1 ,(len(alpha), NO_SIMS))
#alpha =  np.concatenate( (alpha, sims), axis = 1 )
#colnames = ['i']+ [i for i in range(20)]
#alpha = pd.DataFrame(alpha, columns = colnames)
#alpha = data[['i']].merge(alpha, how='left', on='i', right_index=False, suffixes=('', '_sim') )
#alpha = np.array(alpha)[:,1:]

In [11]:
def shape_data(x):
    """ format data to make working with it easier"""
    x = np.array([x])
    x = x.reshape(I,1,T)
    x = np.tile(x ,(1,NSIM,1)).transpose() 
    return x

print shape_data(data['y_t']).shape

(19, 30, 100)


In [18]:
#fix alpha ahead of time

#global variables

class part_d(GenericLikelihoodModel):
    """class for evaluating question 3 part d"""
    
    def __init__(self, sims, *args, **kwds):
        # set appropriate counts for moment conditions and parameters
        super(part_d, self).__init__(*args, **kwds)
        self.sims = sims
    
    def nloglikeobs(self, params, v=False):
        t0, t1, t2, sigma = params
        y = shape_data(self.endog)
        
        p = shape_data(self.exog.transpose()[0])
        y_lag = shape_data(self.exog.transpose()[1])
        alpha = self.sims
        
        #calculate the mean 'delta' for the inside good
        U1 = np.exp(t0 + t1*p + t2*y_lag + sigma*alpha)
     
        #calculate ll, for each simulation
        like =  y*U1/(1+U1) + (1-y)/(1+U1)
        like =  1./NSIM * (like.prod(axis=0)).sum(axis=0)
        like = np.log(like).sum(axis = 0)

        if v: raise Exception('Stop drop and roll')
        return - like

    
    def fit(self, start_params=None, maxiter=1000, maxfun=5000, **kwds):
        # we have one additional parameter and we need to add it for summary
        if start_params == None:
            start_params = [-.798,.5,.5,.5]
        return super(part_d, self).fit(start_params=start_params,
                                       maxiter=maxiter, maxfun=maxfun, **kwds)
    

model_d = part_d(alpha, data['y_t'] ,data[['p_t','y_t-1']])

result_d = model_d.fit()
print(result_d.summary(xname=['theta_0', 'theta_1', 'theta_2', 'sigma']))

Optimization terminated successfully.
         Current function value: 0.622177
         Iterations: 113
         Function evaluations: 200
                                part_d Results                                
Dep. Variable:                    y_t   Log-Likelihood:                -1182.1
Model:                         part_d   AIC:                             2368.
Method:            Maximum Likelihood   BIC:                             2379.
Date:                Sun, 04 Nov 2018                                         
Time:                        14:58:54                                         
No. Observations:                1900                                         
Df Residuals:                    1898                                         
Df Model:                           1                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------

# Part e

# Part f

In [88]:
class part_f(GenericLikelihoodModel):
    """class for evaluating question 3 part f"""
    
    def nloglikeobs(self, params, v=False):
        t0, t1, t2 = params
        y = self.endog
        p,y_lag = self.exog.transpose()
    
        #calculate the mean 'delta' for the inside good
        U1 = t0 + t1*p + t2*y_lag
        U1 = np.exp(U1)
        
        #calculate ll, for each simulation
        likelihood_sims = np.log(y*U1/(1+U1) + (1-y)/(1+U1))
        likelihood = likelihood_sims.sum(axis=0)
        
        if v: raise Exception('Stop drop and roll')
        return - likelihood.sum()

    
    def fit(self, start_params=None, maxiter=1000, maxfun=5000, **kwds):
        # we have one additional parameter and we need to add it for summary
        if start_params == None:
            start_params = [-.948,.5,.5]
        return super(part_f, self).fit(start_params=start_params,
                                       maxiter=maxiter, maxfun=maxfun, **kwds)
    

model_f = part_f(data['y_t'],data[['p_t','y_t-1']])

result_f = model_f.fit()
print(result_f.summary(xname=['theta_0', 'theta_1', 'theta_2']))

Optimization terminated successfully.
         Current function value: 0.643884
         Iterations: 84
         Function evaluations: 148
                                part_f Results                                
Dep. Variable:                    y_t   Log-Likelihood:                -1223.4
Model:                         part_f   AIC:                             2451.
Method:            Maximum Likelihood   BIC:                             2462.
Date:                Thu, 25 Oct 2018                                         
Time:                        16:31:57                                         
No. Observations:                1900                                         
Df Residuals:                    1898                                         
Df Model:                           1                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------

# Part g